# 21. Asynchronous Programming

## A Few Definitions

_Native coroutine_

A coroutine function defined with `async def`. You can delegate from a native coroutine to another native coroutine using the `await` keyword, similar to how classic coroutines use `yield from`. The `async def` statement always defines a native coroutine, even if the `await` keyword is not used in its body. The `await` keyword cannot be used outside of a native coroutine.

_Classic coroutine_

A generator function that consumes data sent to it via `my_coro.send(data)` calls, and reads that data by using `yield` in an expression. Classic coroutines can delegate to other classic coroutines using `yield from`. Classic coroutines cannot be driven by `await`, and are no longer supported by _asyncio_.

_Generator-based coroutine_

A generator function decorated with `@types.coroutine`—introduced in Python 3.5. That decorator makes the generator compatible with the new `await` keyword.

_Asynchronous generator_



## An `asyncio` Example: Probing Domains

Imagine you are about to start a new blog on Python, and you plan to register a domain using a Python keyword and the **.DEV** suffix—for example: `AWAIT.DEV`. Example following is a script using asyncio to check several domains concurrently.

```python
#!/usr/bin/env python3

import asyncio
import socket

from keyword import kwlist

MAX_KEYWORD_LEN = 4

async def probe(domain: str) -> tuple[str, bool]:
    loop = asyncio.get_running_loop()
    try:
        await loop.getaddrinfo(domain, None)
    except socket.gaierror:
        return (domain, False)
    return (domain, True)

async def main() -> None:
    names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN)
    domains = (f'{name}.dev'.lower() for name in names)
    coros = [probe(domain) for domain in domains]
    for coro in asyncio.as_completed(coros):
        domain, found = await coro
        mark = '+' if found else ' '
        print(f'{mark} {domain}')

if __name__ == '__main__':
    asyncio.run(main())
```

**Results**:

```text
→ ./blogdom.py 
  pass.dev
  with.dev
  if.dev
+ and.dev
  try.dev
+ true.dev
+ for.dev
+ def.dev
  none.dev
+ elif.dev
  is.dev
  or.dev
+ in.dev
+ del.dev
+ from.dev
+ as.dev
  else.dev
+ not.dev
```

## New Concept: Awaitable

The `for` keyword works with _iterables_. The `await` keyword works with _awaitables_.

As an end user of _asyncio_, these are the awaitables you will see on a daily basis:

+ A _native coroutine object_, which you get by calling a _native coroutine function_
+ An `asyncio.Task`, which you usually get by passing a coroutine object to `asyncio.create_task()`

However, end-user code does not always need to await on a `Task`. We use `asyncio.create_task(one_coro())` to schedule `one_coro` for concurrent execution, without waiting for its return. If you don’t expect to cancel the task or wait for it, there is no need to keep the `Task` object returned from `create_task`. Creating the task is enough to schedule the coroutine to run.

In contrast, we use `await other_coro()` to run `other_coro` right now and wait for its completion because we need its result before we can proceed. In _spinner_async.p_y, the supervisor coroutine did `res = await slow()` to execute `slow` and get its result.

## Downloading with `asyncio` and HTTPX

The _flags_asyncio.py_ script downloads a fixed set of 20 flags from _fluentpython.com_.

As of Python 3.10, asyncio only supports TCP and UDP directly, and there are no asynchronous HTTP client or server packages in the standard library. I am using HTTPX in all the HTTP client examples.

```python

#!/usr/bin/env python3

"""Download flags of top 20 countries by population

asyncio + aiottp version

Sample run::

    $ python3 flags_asyncio.py
    EG VN IN TR RU ID US DE CN MX JP BD NG ET FR BR PH PK CD IR
    20 flags downloaded in 1.07s
"""

import asyncio

from httpx import AsyncClient  # <1>

from flags import BASE_URL, save_flag, main  # <2>

async def download_one(client: AsyncClient, cc: str):  # <3>
    image = await get_flag(client, cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

async def get_flag(client: AsyncClient, cc: str) -> bytes:  # <4>
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = await client.get(url, timeout=6.1,
                                  follow_redirects=True)  # <5>
    return resp.read()  # <6>

def download_many(cc_list: list[str]) -> int:    # <1>
    return asyncio.run(supervisor(cc_list))      # <2>

async def supervisor(cc_list: list[str]) -> int:
    async with AsyncClient() as client:          # <3>
        to_do = [download_one(client, cc)
                 for cc in sorted(cc_list)]      # <4>
        res = await asyncio.gather(*to_do)       # <5>

    return len(res)                              # <6>

if __name__ == '__main__':
    main(download_many)

````


**Output**:

```text
╰→ ./flags_asyncio.py 
BD TR JP VN CN ID NG RU EG DE FR IN BR PK CD PH IR ET US MX 
20 downloads in 1.00s

╰→ ./flags_asyncio.py 
BR NG IN TR VN BD FR CN DE ID EG RU JP PH ET US PK IR CD MX 
20 downloads in 0.86s
````


## The Secret of Native Coroutines: Humble Generators

A key difference between the classic coroutine examples we saw in _Classic Coroutines_ and _flags_asyncio.py_ is that there are no visible `.send()` calls or `yield` expressions in the latter.


### The All-or-Nothing Problem

For peak performance with `asyncio`, we must replace every function that does I/O with an asynchronous version that is activated with `await` or `asyncio.create_task`, so that control is given back to the event loop while the function waits for I/O. If you can’t rewrite a blocking function as a coroutine, you should run it in a separate thread or process.

## Asynchronous Context Managers

Consider the _asyncio_-compatible PostgreSQL driver:

```python
tr = connection.transaction()
await tr.start()
try:
    await connection.execute("INSERT INTO mytable VALUES (1,2,3))")
except:
    await tr.rollback()
    raise
else:
    await tr.commit()
```

With `async with` the exampple can be written:
```python
async with connection.transaction():
    await connection.execute("INSERT INTO mytable VALUES (1,2,3))"
````


## Enhancing the `asyncio` Downloader

```python
